In [1]:
#load library
import pandas as pd
import numpy as np
import tensorflow 
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('/content/df_preprocessed.csv')

In [3]:
df

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,label,damage_category,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
0,7,5,4.468204,26.2,94.3,1.808289,8.2,51,6.7,0,0.0,No damage,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,7,4,4.517431,35.4,669.1,2.041220,18.0,33,0.9,0,0.0,No damage,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,7,4,4.517431,43.7,686.9,2.041220,14.6,33,1.3,0,0.0,No damage,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,8,6,4.529368,33.3,77.5,2.302585,8.3,97,4.0,1,0.0,No damage,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,8,6,4.503137,51.3,102.2,2.360854,11.4,99,1.8,0,0.0,No damage,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,4,3,4.414010,56.7,665.6,1.064711,27.8,32,2.7,0,1.0,moderate,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
506,2,4,4.414010,56.7,665.6,1.064711,21.9,71,5.8,0,1.0,high,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
507,7,4,4.414010,56.7,665.6,1.064711,21.2,70,6.7,0,1.0,moderate,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
508,1,4,4.558079,146.0,614.7,2.509599,25.6,42,4.0,0,0.0,No damage,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [4]:
df.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,label,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
count,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000
mean,4.680392,4.294118,4.522609,111.837647,550.470392,2.228457,18.951569,44.029412,4.017255,0.015686,0.527451,0.143137,0.158824,0.182353,0.119608,0.125490,0.105882,0.358824,0.017647,0.037255,0.001961,0.062745,0.031373,0.103922,0.003922,0.001961,0.029412,0.333333
std,2.320534,1.234636,0.034979,63.853719,246.630662,0.428759,5.789930,15.968323,1.788793,0.124381,0.499736,0.350557,0.365870,0.386514,0.324821,0.331599,0.307989,0.480126,0.131794,0.189572,0.044281,0.242742,0.174494,0.305458,0.062561,0.044281,0.169124,0.471867
min,1.000000,2.000000,4.332048,3.000000,7.900000,0.741937,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,4.000000,4.514151,73.250000,442.300000,2.041220,15.625000,32.250000,2.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,4.000000,4.528829,108.400000,664.200000,2.240710,19.300000,41.500000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.000000,5.000000,4.542230,142.400000,714.200000,2.484907,22.800000,53.000000,4.900000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.000000,9.000000,4.576771,291.300000,860.600000,4.044804,33.300000,99.000000,9.400000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
# def remove_outlier_iqr(df):
#   Q1= df.quantile(0.25)
#   Q3= df.quantile(0.75)
#   IQR =Q3-Q1
#   df_final = df[~((df<(Q1-1.5*IQR))|(df>(Q3+1.5*IQR)))]
#   return df_final
# df_removed_outlier = remove_outlier_iqr(df[['FFMC','DMC','RH']])
# df = df[df_removed_outlier['FFMC'].notna()]
# df.dropna(inplace=True)

In [6]:
df.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,label,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
count,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000
mean,4.680392,4.294118,4.522609,111.837647,550.470392,2.228457,18.951569,44.029412,4.017255,0.015686,0.527451,0.143137,0.158824,0.182353,0.119608,0.125490,0.105882,0.358824,0.017647,0.037255,0.001961,0.062745,0.031373,0.103922,0.003922,0.001961,0.029412,0.333333
std,2.320534,1.234636,0.034979,63.853719,246.630662,0.428759,5.789930,15.968323,1.788793,0.124381,0.499736,0.350557,0.365870,0.386514,0.324821,0.331599,0.307989,0.480126,0.131794,0.189572,0.044281,0.242742,0.174494,0.305458,0.062561,0.044281,0.169124,0.471867
min,1.000000,2.000000,4.332048,3.000000,7.900000,0.741937,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,4.000000,4.514151,73.250000,442.300000,2.041220,15.625000,32.250000,2.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,4.000000,4.528829,108.400000,664.200000,2.240710,19.300000,41.500000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.000000,5.000000,4.542230,142.400000,714.200000,2.484907,22.800000,53.000000,4.900000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.000000,9.000000,4.576771,291.300000,860.600000,4.044804,33.300000,99.000000,9.400000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
df.drop('damage_category',axis=1,inplace=True)

In [8]:
# Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [9]:
calc_vif(df.drop(['label','FFMC','DC','ISI','temp','month_sep','month_aug'],axis=1))

,variables,VIF
0,X,7.623238
1,Y,15.864570
2,DMC,6.486783
3,RH,8.786094
4,wind,6.042955
5,rain,1.101271
6,day_mon,1.765099
7,day_sat,1.831676
8,day_sun,1.984669
9,day_thu,1.620317


In [10]:
df.drop(['FFMC','DC','ISI','temp','month_sep','month_aug'],axis=1,inplace=True)

In [11]:
df

,X,Y,DMC,RH,wind,rain,label,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct
0,7,5,26.2,51,6.7,0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,7,4,35.4,33,0.9,0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,7,4,43.7,33,1.3,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,8,6,33.3,97,4.0,1,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,8,6,51.3,99,1.8,0,0.0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,4,3,56.7,32,2.7,0,1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
506,2,4,56.7,71,5.8,0,1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
507,7,4,56.7,70,6.7,0,1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
508,1,4,146.0,42,4.0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
features = df.drop(columns='label')
labels = df['label']
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.8, random_state=20)

In [13]:
predictor_vars = X_train.columns

for col in predictor_vars:
    # Calculating variable mean and std from training data
    col_mean = X_train[col].mean()
    col_std = X_train[col].std()
    if col_std == 0:
        col_std = 1e-20
    X_train[col] = (X_train[col] - col_mean) / col_std
    X_test[col] = (X_test[col] - col_mean) / col_std    

In [14]:
X_train=X_train.to_numpy()
X_test=X_test.to_numpy()
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()

In [15]:
# define the keras sequential model
model = Sequential()

In [16]:
model.add(Dense(50, activation='relu')) # input layer + 1st hidden layer
model.add(Dense(30, activation='relu')) # 2nd hidden layer 
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # output layerm

In [17]:
  # compile model
  optimizer = SGD(lr=0.01, momentum=0.9)
  model.compile(loss='hinge', optimizer=optimizer, metrics=['accuracy'])
  _, train_acc = model.evaluate(X_train,y_train, verbose=0)
  _, valid_acc = model.evaluate(X_test,y_test, verbose=0)
  print(train_acc)
  print(valid_acc)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


0.5784313678741455
0.5024510025978088


In [18]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(neurons=1):
	# create model
	model = Sequential()
	model.add(Dense(neurons, input_dim=21, kernel_initializer='uniform', activation='softmax', kernel_constraint=maxnorm(4)))
	model.add(Dropout(0.2))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train,y_train, validation_data=(X_test,y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.


Best: 0.647059 using {'neurons': 5}
0.627451 (0.049990) with: {'neurons': 1}
0.647059 (0.072044) with: {'neurons': 5}
0.647059 (0.072044) with: {'neurons': 10}
0.637255 (0.060435) with: {'neurons': 15}
0.647059 (0.072044) with: {'neurons': 20}
0.637255 (0.060435) with: {'neurons': 25}
0.647059 (0.072044) with: {'neurons': 30}


In [19]:
# Function to create model, required for KerasClassifier
def create_model(activation='relu'):
	# create model
	model = Sequential()
	model.add(Dense(10, input_dim=21, kernel_initializer='uniform', activation=activation))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train,y_train, validation_data=(X_test,y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  from ipykernel import kernelapp as app


Best: 0.647059 using {'activation': 'softmax'}
0.647059 (0.041595) with: {'activation': 'softmax'}
0.637255 (0.036683) with: {'activation': 'softplus'}
0.607843 (0.036683) with: {'activation': 'softsign'}
0.617647 (0.041595) with: {'activation': 'relu'}
0.607843 (0.036683) with: {'activation': 'tanh'}
0.637255 (0.060435) with: {'activation': 'sigmoid'}
0.647059 (0.072044) with: {'activation': 'hard_sigmoid'}
0.617647 (0.024015) with: {'activation': 'linear'}


In [20]:

# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
	# create model
	model = Sequential()
	model.add(Dense(10, input_dim=21, activation='softmax'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train,y_train, validation_data=(X_test,y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.


Best: 0.676471 using {'optimizer': 'Adamax'}
0.637255 (0.055459) with: {'optimizer': 'SGD'}
0.637255 (0.084337) with: {'optimizer': 'RMSprop'}
0.490196 (0.113489) with: {'optimizer': 'Adagrad'}
0.441176 (0.149971) with: {'optimizer': 'Adadelta'}
0.647059 (0.086586) with: {'optimizer': 'Adam'}
0.676471 (0.024015) with: {'optimizer': 'Adamax'}
0.627451 (0.073366) with: {'optimizer': 'Nadam'}


In [21]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
	# create model
	model = Sequential()
	model.add(Dense(10, input_dim=21, kernel_initializer='uniform', activation='softmax', kernel_constraint=maxnorm(4)))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Compile model
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train,y_train, validation_data=(X_test,y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Best: 0.647059 using {'learn_rate': 0.1, 'momentum': 0.4}
0.627451 (0.049990) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.627451 (0.049990) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.627451 (0.049990) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.627451 (0.049990) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.627451 (0.049990) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.627451 (0.049990) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.627451 (0.049990) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.627451 (0.049990) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.627451 (0.049990) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.627451 (0.049990) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.627451 (0.049990) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.627451 (0.049990) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.627451 (0.049990) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.637255 (0.060435) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.647059 (0.063537) with: {'learn_rate': 0

In [22]:
# define the keras sequential model
model = Sequential()

In [23]:
model.add(Dense(10, input_dim=21, kernel_initializer='uniform', activation='softmax', kernel_constraint=maxnorm(4)))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

In [24]:
# compile model
optimizer = SGD(learning_rate=0.1, momentum=0.2)
model.compile(loss='hinge', optimizer=optimizer, metrics=['accuracy'])

In [25]:
history = model.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=100,batch_size=10,verbose=1)

Epoch 1/100
11/11 [==============================] - 1s 23ms/step - loss: 0.8700 - accuracy: 0.6275 - val_loss: 0.9974 - val_accuracy: 0.5025
Epoch 2/100
11/11 [==============================] - 0s 10ms/step - loss: 0.8641 - accuracy: 0.6275 - val_loss: 0.9973 - val_accuracy: 0.5025
Epoch 3/100
11/11 [==============================] - 0s 10ms/step - loss: 0.8570 - accuracy: 0.6275 - val_loss: 0.9972 - val_accuracy: 0.5025
Epoch 4/100
11/11 [==============================] - 0s 10ms/step - loss: 0.8496 - accuracy: 0.6275 - val_loss: 0.9971 - val_accuracy: 0.5025
Epoch 5/100
11/11 [==============================] - 0s 10ms/step - loss: 0.8462 - accuracy: 0.6275 - val_loss: 0.9970 - val_accuracy: 0.5025
Epoch 6/100
11/11 [==============================] - 0s 10ms/step - loss: 0.8410 - accuracy: 0.6275 - val_loss: 0.9969 - val_accuracy: 0.5025
Epoch 7/100
11/11 [==============================] - 0s 10ms/step - loss: 0.8346 - accuracy: 0.6275 - val_loss: 0.9968 - val_accuracy: 0.5025
Epoch 

In [26]:
# evaluate the model
_, train_acc = model.evaluate(X_train,y_train, verbose=0)
_, valid_acc = model.evaluate(X_test,y_test, verbose=0)
print('Train: %.3f, Valid: %.3f' % (train_acc, valid_acc))

Train: 0.627, Valid: 0.502


In [28]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(10, input_dim=21, kernel_initializer='uniform', activation='softmax', kernel_constraint=maxnorm(4)))
	model.add(Dense(1, activation='sigmoid'))
 
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.1, momentum=0.2), metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
epochs = [100,200,300,400,500,600,700,800,900,1000]
batch_size = [1,32,128,256,510]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train,y_train, validation_data=(X_test,y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.


Best: 0.656863 using {'batch_size': 510, 'epochs': 400}
0.598039 (0.084337) with: {'batch_size': 1, 'epochs': 100}
0.549020 (0.060435) with: {'batch_size': 1, 'epochs': 200}
0.598039 (0.084337) with: {'batch_size': 1, 'epochs': 300}
0.578431 (0.132262) with: {'batch_size': 1, 'epochs': 400}
0.588235 (0.063537) with: {'batch_size': 1, 'epochs': 500}
0.598039 (0.077196) with: {'batch_size': 1, 'epochs': 600}
0.588235 (0.063537) with: {'batch_size': 1, 'epochs': 700}
0.588235 (0.024015) with: {'batch_size': 1, 'epochs': 800}
0.627451 (0.060435) with: {'batch_size': 1, 'epochs': 900}
0.588235 (0.063537) with: {'batch_size': 1, 'epochs': 1000}
0.647059 (0.072044) with: {'batch_size': 32, 'epochs': 100}
0.627451 (0.027730) with: {'batch_size': 32, 'epochs': 200}
0.617647 (0.048029) with: {'batch_size': 32, 'epochs': 300}
0.578431 (0.013865) with: {'batch_size': 32, 'epochs': 400}
0.617647 (0.024015) with: {'batch_size': 32, 'epochs': 500}
0.578431 (0.027730) with: {'batch_size': 32, 'epochs'